# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 0
- firstName of user1
- gender of user 2
- item number in session 3
- last name of user 4
- length of the song 5
- level (paid or free song) 6
- location of the user 7
- sessionId 8
- song title 9
- userId 10

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

# Now we will create three tables to run the queries:
## songs_by_user_session Table:
####   For this table, the sessionid is used as a partition key because the queries will filter by this column. 
####   iteminsession is used as clustering column to help make up a unique key and sort data.

In [9]:
query = "CREATE TABLE IF NOT EXISTS songs_by_user_session "
query = query + "(sessionid int, , iteminsession int, artist text, song_title text, song_length float, \
                  PRIMARY KEY (sessionid, iteminsession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

## songs_by_userid Table:
####   For this table, the userid and sessionid is used as a partition key (Composite key) because the queries will filter by this column. 
####   iteminsession is used as clustering column to help make up a unique key and sort data.

In [10]:
query = "CREATE TABLE IF NOT EXISTS songs_by_userid "
query = query + "(userid int, sessionid int, iteminsession int, artist text, song_title text, user_fname text, user_lname text, \
                  PRIMARY KEY ((userid, sessionid), iteminsession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

## users_by_song Table:
####   For this table, the song_title is used as a partition key because the queries will filter by this column. 
####   userid is used as clustering column to help make up a unique key and sort data.


In [11]:
query = "CREATE TABLE IF NOT EXISTS users_by_song "
query = query + "(song_title text, userId int, user_fname text, user_lname text, \
                  PRIMARY KEY (song_title, userid))"
try:
    session.execute(query)
except Exception as e:
    print(e)

# Insert data into First Table Songs_by_user_session

In [12]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header 
    for line in csvreader:  
        query = "INSERT INTO songs_by_user_session (sessionid, iteminsession, artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, ( int(line[8]), int(line[3]), line[0], line[9], float(line[5])))        

# Insert data into Second Table songs_by_userid

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_by_userid (userid, sessionid, iteminsession, artist, song_title, user_fname , user_lname)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))        

# Insert data into Third Table users_by_song

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header (song_title text, userId int, user_fname text, user_lname text,
    for line in csvreader:
        query = "INSERT INTO users_by_song (song_title, userid, user_fname, user_lname)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))       

# Query 1:
## Give me the artist, song title and song's length in the music app history that was heard during       sessionId = 338, and itemInSession = 4

In [15]:
query = "SELECT artist, song_length, song_title FROM songs_by_user_session WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_length, row.song_title)
      

Faithless 495.30731201171875 Music Matters (Mark Knight Dub)


# Query 2:
## Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [16]:
query = "SELECT artist, song_title, user_fname, user_lname FROM songs_by_userid WHERE userid = 10 AND sessionid = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.user_fname, row.user_lname, row.song_title)


Down To The Bone Sylvie Cruz Keep On Keepin' On
Three Drives Sylvie Cruz Greece 2000
Sebastien Tellier Sylvie Cruz Kilometer
Lonnie Gordon Sylvie Cruz Catch You Baby (Steve Pitron & Max Sanna Radio Edit)


# Query 3:
## Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [17]:
query = "SELECT user_fname, user_lname FROM users_by_song WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_fname, row.user_lname)

                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [18]:
query = "drop table Songs_by_user_session"
query1 = "drop table songs_by_userid"
query2 = "drop table users_by_song"
try:
    row = session.execute(query)
    row1 = session.execute(query1)
    row2 = session.execute(query2)

except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()